In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

from time import time
from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df
from utils.preprocessing import inverse_dummy
from sklearn.model_selection import train_test_split
from utils.preprocessing import preprocess_df
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.cf_proto import  generate_cf_proto_result, process_result

from alibi.explainers import CounterFactualProto
from alibi_cf.utils import get_cat_vars_dict
from utils.save import save_result_as_csv

tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  False


In [2]:
dataset_name = 'diabetes' # [adult, german, compas]

if dataset_name == 'adult':
    dataset_loading_fn = load_adult_df
elif dataset_name == 'german':
    dataset_loading_fn = load_german_df
elif dataset_name == 'compas':
    dataset_loading_fn = load_compas_df
elif dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
else:
    raise Exception("Unsupported dataset")

In [3]:
df_info = preprocess_df(dataset_loading_fn)

In [4]:
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [5]:
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [6]:
### Train models
models = train_three_models(X_train, y_train)
save_three_models(models, dataset_name)

Train on 614 samples
Epoch 1/20
614/614 [==============================] - 0s 119us/sample - loss: 0.6984 - acc: 0.4055
Epoch 2/20
614/614 [==============================] - 0s 28us/sample - loss: 0.6878 - acc: 0.6450
Epoch 3/20
614/614 [==============================] - 0s 18us/sample - loss: 0.6835 - acc: 0.6580
Epoch 4/20
614/614 [==============================] - 0s 22us/sample - loss: 0.6784 - acc: 0.6580
Epoch 5/20
614/614 [==============================] - 0s 24us/sample - loss: 0.6729 - acc: 0.6580
Epoch 6/20
614/614 [==============================] - 0s 37us/sample - loss: 0.6667 - acc: 0.6580
Epoch 7/20
614/614 [==============================] - 0s 16us/sample - loss: 0.6586 - acc: 0.6580
Epoch 8/20
614/614 [==============================] - 0s 33us/sample - loss: 0.6502 - acc: 0.6580
Epoch 9/20
614/614 [==============================] - 0s 26us/sample - loss: 0.6429 - acc: 0.6580
Epoch 10/20
614/614 [==============================] - 0s 15us/sample - loss: 0.6361 - acc: 0.65

In [7]:
models = load_three_models(X_train.shape[-1], dataset_name)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [8]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.7727] | RF [0.7987] | NN [0.7662]


# Alibi

## 1. Counterfactual Prototype

In [9]:
num_instances = 5
num_cf_per_instance = 1

In [10]:
results = generate_cf_proto_result(df_info, train_df, models, num_instances, num_cf_per_instance, X_train, X_test, y_test, max_iters=500)
result_dfs = process_result(results, df_info)

Finding counterfactual for dt
instance 0
CF 0
Found CF
instance 1
CF 0
Found CF
instance 2
CF 0
Found CF
instance 3
CF 0
CF not found
instance 4
CF 0
CF not found
Finding counterfactual for rfc
instance 0
CF 0
Found CF
instance 1
CF 0
CF not found
instance 2
CF 0
Found CF
instance 3
CF 0
CF not found
instance 4
CF 0
CF not found
Finding counterfactual for nn
instance 0
CF 0
Found CF
instance 1
CF 0
CF not found
instance 2
CF 0
CF not found
instance 3
CF 0
CF not found
instance 4
CF 0
CF not found


In [11]:
save_result_as_csv("proto", dataset_name, result_dfs)

Result has been saved to ./results/proto_diabetes
